In [547]:
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import trajectory_analysis.trajectory_vectors as trajectory_vectors
import data_extraction.get_indices as get_indices

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Generic starting point for work on session data

In [ ]:
data_folder = data_strings.DATA_FOLDER
json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [549]:
json_filenames = json_filenames_all_social

In [550]:
df, trial_list = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\fourth_experiments_2410\241017\2024-10-17_16-41-38_ZimoElsaSocial.json
Loading complete.
Preprocessing complete.


In [551]:
_, trial_lists = prepare_data.prepare_data(data_folder, data_strings.JSON_FILENAMES_SOCIAL[:2], combine=False)

filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\first_experiments_2409\240913\2024-09-13_11-31-00_YansuJerrySocial.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: D:\Users\Tom\OneDrive\PhD\SWC\data\second_experiments_2409\240927\2024-09-27_14-25-20_SaraEmilySocial.json
Loading complete.
Preprocessing complete.


In [552]:
def player_direct_distance_trial(trial, player_index):
    ''' Return the direct distance in units between a slice onset location 
        and the final location for a single player.
        Takes a trial and player index. '''
    
    # get the player's trajectory
    trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=player_index, debug=False)

    # find distance between first and last trajectory points
    end_position = trajectory[:,-1]
    start_position = trajectory[:,0]
    distance = np.linalg.norm(end_position - start_position)

    return distance

In [553]:
def player_actual_distance_trial(trial, player_index, debug=False):
    ''' Return the actual distance in units throughout the player's entire
        trajectory.
        Takesa trial and player index. '''
    
    # get the player's trajectory
    trajectory = trajectory_vectors.extract_trial_player_trajectory(trial=trial, player_id=player_index, debug=False)

    # compute difference between consecutive columns (timepoints)
    position_diffs = np.diff(trajectory, axis=1) # shape 2, timepoints-1
    if debug:
        print(f"shape of np.diff output: {position_diffs.shape}")

    # square these differences, and sum across x and y for each column (timepoint)
    squared_distances = np.sum(position_diffs**2, axis=0) # shape timepoints-1,

    # take the square root to get the euclidean difference between each consecutive point
    distances = np.sqrt(squared_distances)

    if debug:
        print(f" shape of summed euclidean distance: {distances.shape}")
        print(f"summed euclidean distance[:5]: {distances[:5]}")

    return np.sum(distances)

In [554]:
def player_direct_distance_session(trial_list, player_index):
    ''' Return an array of len(trial_list) of the direct distance between a player's
        start and end location on each trial.
        Takes a list of trials. '''
    

    direct_distances = np.zeros(len(trial_list))
    for i, trial in enumerate(trial_list):
        trial = trial_list[i]
        distance = player_direct_distance_trial(trial, player_index)

        direct_distances[i] = distance

    return direct_distances


In [555]:
def player_actual_distance_session(trial_list, player_index):
    ''' Return an array of len(trial_list) of the direct distance between a player's
        start and end location on each trial.
        Takes a list of trials. '''
    

    actual_distances = np.zeros(len(trial_list))
    for i, trial in enumerate(trial_list):
        trial = trial_list[i]
        distance = player_actual_distance_trial(trial, player_index)

        actual_distances[i] = distance

    return actual_distances

In [556]:
def direct_distance_winner_loser_session(trial_list):
    ''' Return 4 arrays, each of len(trial_list), which are the direct distances in
        winning trials and losing trials respectively for player 0 and player 1 respectively.
        Indices where the trial outcome does match the array name are np.nan. '''
    
    # fill all arrays with the direct distances for the player
    player_0_win_direct_distances = player_direct_distance_session(trial_list, player_index=0)
    player_0_loss_direct_distances = player_0_win_direct_distances
    
    player_1_win_direct_distances = player_direct_distance_session(trial_list, player_index=1)
    player_1_loss_direct_distances = player_1_win_direct_distances

    # find the winners in this session
    winners_session = get_indices.get_trigger_activators(trial_list)
    losers_session = (winners_session-1)*-1

    # replace array elements with np.nan at indices where the trial outcome does not match the array name 
    player_0_win_direct_distances = np.where(winners_session == 0, player_0_win_direct_distances, np.nan)
    player_0_loss_direct_distances = np.where(losers_session == 0, player_0_loss_direct_distances, np.nan)

    player_1_win_direct_distances = np.where(winners_session == 1, player_1_win_direct_distances, np.nan)
    player_1_loss_direct_distances = np.where(losers_session == 1, player_1_loss_direct_distances, np.nan)

    return (player_0_win_direct_distances, player_0_loss_direct_distances, player_1_win_direct_distances,
            player_1_loss_direct_distances)



In [557]:
def actual_distance_winner_loser_session(trial_list):
    ''' Return 4 arrays, each of len(trial_list), which are the direct distances in
        winning trials and losing trials respectively for player 0 and player 1 respectively.
        Indices where the trial outcome does match the array name are np.nan. '''
    
    player_0_win_actual_distances = player_actual_distance_session(trial_list, player_index=0)
    player_0_loss_actual_distances = player_0_win_actual_distances
    
    player_1_win_actual_distances = player_actual_distance_session(trial_list, player_index=1)
    player_1_loss_actual_distances = player_1_win_actual_distances

        # find the winners in this session
    winners_session = get_indices.get_trigger_activators(trial_list)
    losers_session = (winners_session-1)*-1

    # replace array elements with np.nan at indices where the trial outcome does not match the array name 
    player_0_win_actual_distances = np.where(winners_session == 0, player_0_win_actual_distances, np.nan)
    player_0_loss_actual_distances = np.where(losers_session == 0, player_0_loss_actual_distances, np.nan)

    player_1_win_actual_distances = np.where(winners_session == 1, player_1_win_actual_distances, np.nan)
    player_1_loss_actual_distances = np.where(losers_session == 1, player_1_loss_actual_distances, np.nan)

    return (player_0_win_actual_distances, player_0_loss_actual_distances, player_1_win_actual_distances,
            player_1_loss_actual_distances)


In [558]:
def ratio_direct_to_absolute_distances_session(trial_list):
    ''' Return 4 values, the ratio of direct distances to absolute distances for 
        winner and then loser for first player 0 and then player 1 '''
    

    # find the actual distances for both players, win and loss
    (player_0_win_actual_distances, player_0_loss_actual_distances, player_1_win_actual_distances,
                player_1_loss_actual_distances) = actual_distance_winner_loser_session(trial_list)

    # find the direct distances for both players, win and loss
    (player_0_win_direct_distances, player_0_loss_direct_distances, player_1_win_direct_distances,
                player_1_loss_direct_distances) = direct_distance_winner_loser_session(trial_list)
    
    # calculate ratios
    ratio_player_0_win =  np.nanmean(player_0_win_direct_distances/player_0_win_actual_distances)
    ratio_player_0_loss =  np.nanmean(player_0_loss_direct_distances/player_0_loss_actual_distances)
    ratio_player_1_win = np.nanmean(player_1_win_direct_distances/player_1_win_actual_distances)
    ratio_player_1_loss =  np.nanmean(player_1_loss_direct_distances/player_1_loss_actual_distances)

    return (ratio_player_0_win, ratio_player_0_loss, ratio_player_1_win, ratio_player_1_loss)

In [559]:
def ratio_direct_to_absolute_distances_multiple_sessions(trial_lists):
    ''' Takes a list of trial lists (from multiple sessions) and returns
        a (4,num_sessions) array of trajectory effiency ratios.
        Columns are: player_0_win, player_0_loss, player_1_win, player_1_loss. '''
    
    trajectory_efficiency_ratios = np.zeros((len(trial_lists),4))

    for i, trial_list in enumerate(trial_lists):
        
        (ratio_player_0_win, ratio_player_0_loss,
        ratio_player_1_win, ratio_player_1_loss) = ratio_direct_to_absolute_distances_session(trial_list)

        trajectory_efficiency_ratios[i,:] = (ratio_player_0_win, ratio_player_0_loss,
                                            ratio_player_1_win, ratio_player_1_loss)
        
    return trajectory_efficiency_ratios

In [560]:
(ratio_player_0_win, ratio_player_0_loss, ratio_player_1_win, ratio_player_1_loss) = ratio_direct_to_absolute_distances_session(trial_list)

C:\Users\Tom\AppData\Local\Temp\ipykernel_16692\72825313.py:16: RuntimeWarning: invalid value encountered in divide
  ratio_player_0_loss =  np.nanmean(player_0_loss_direct_distances/player_0_loss_actual_distances)


In [561]:
trajectory_efficiency_ratios = ratio_direct_to_absolute_distances_multiple_sessions(trial_lists)

C:\Users\Tom\AppData\Local\Temp\ipykernel_16692\72825313.py:16: RuntimeWarning: invalid value encountered in divide
  ratio_player_0_loss =  np.nanmean(player_0_loss_direct_distances/player_0_loss_actual_distances)


In [562]:
trajectory_efficiency_ratios

array([[0.81805436, 0.75260653, 0.76958186, 0.62714763],
       [0.8528626 , 0.74524714, 0.97358384, 0.97010905]])

In [563]:
ratio_player_0_win, ratio_player_0_loss, ratio_player_1_win, ratio_player_1_loss

(np.float64(0.9196845327488561),
 np.float64(0.7639445690966866),
 np.float64(0.9753591794777994),
 np.float64(0.9292355321249816))

In [564]:
# find the actual distances for both players, win and loss
(player_0_win_actual_distances, player_0_loss_actual_distances, player_1_win_actual_distances,
            player_1_loss_actual_distances) = actual_distance_winner_loser_session(trial_list)

# find the direct distances for both players, win and loss
(player_0_win_direct_distances, player_0_loss_direct_distances, player_1_win_direct_distances,
            player_1_loss_direct_distances) = direct_distance_winner_loser_session(trial_list)

In [565]:
np.nanmean(player_0_win_direct_distances)/np.nanmean(player_0_win_actual_distances)

np.float64(0.9129882309442566)

In [566]:
np.nanmean(player_1_win_direct_distances)/np.nanmean(player_1_win_actual_distances)

np.float64(0.9694372187316416)

In [567]:
a = player_0_win_direct_distances
b = player_0_win_actual_distances

a < b

array([False,  True, False,  True,  True,  True, False,  True,  True,
       False, False,  True, False, False, False, False, False, False,
        True, False, False,  True, False, False,  True,  True, False,
       False, False, False, False, False,  True,  True, False, False,
        True, False,  True,  True, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False,  True,
       False,  True, False,  True, False, False, False, False,  True,
        True, False, False, False, False,  True, False,  True, False,
       False, False, False, False,  True,  True,  True, False,  True,
       False, False, False,  True,  True, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False,  True,  True, False, False, False,  True,
        True, False, False, False, False,  True, False, False, False,
        True, False,

In [568]:
player_1_win_direct_distances

array([18.83628049,         nan, 19.39124114,         nan,         nan,
               nan, 19.65127026,         nan,         nan, 19.58822441,
       17.3856904 ,         nan, 19.65467404, 17.46011065, 17.07823738,
       16.52886919, 15.00217567, 19.21148406,         nan, 15.9066032 ,
       17.29733606,         nan, 18.75869673, 17.72241695,         nan,
               nan, 18.89211592, 16.05315725, 17.23221691, 15.70498368,
       17.33184737, 16.78489567,         nan,         nan, 17.9668422 ,
       18.60336421,         nan, 16.43659816,         nan,         nan,
       15.56568245, 18.61522118, 18.64066133, 18.33371295,         nan,
               nan,         nan,         nan,         nan, 16.52205371,
       17.57755177, 16.02019229, 17.35410258,         nan, 15.06871996,
               nan, 17.9288719 ,         nan, 15.54055605, 19.01059391,
       19.01144408, 18.65410502,         nan,         nan, 16.24914003,
       16.69955145, 17.52609366, 18.85706939,         nan, 19.92

In [569]:
player_1_win_actual_distances

array([25.23054504,         nan, 19.84788156,         nan,         nan,
               nan, 21.88668245,         nan,         nan, 19.69963715,
       17.62958675,         nan, 19.73883977, 17.56078181, 17.41536505,
       17.29875991, 34.25761487, 19.21150052,         nan, 16.29158067,
       17.49289669,         nan, 18.87234218, 18.57051333,         nan,
               nan, 19.37908061, 16.31221593, 17.42852628, 15.88266334,
       17.34599464, 16.89924379,         nan,         nan, 18.58218877,
       18.64338854,         nan, 16.83175879,         nan,         nan,
       15.71393152, 18.84286196, 18.64183696, 19.0495324 ,         nan,
               nan,         nan,         nan,         nan, 16.63847067,
       17.69365746, 16.19970049, 17.42649765,         nan, 19.01541053,
               nan, 18.05840619,         nan, 16.76219846, 19.06918079,
       19.26036117, 18.80279411,         nan,         nan, 16.34913597,
       16.70232452, 18.43141608, 19.62001192,         nan, 20.09

In [570]:
np.nanmean(player_0_win_direct_distances/player_0_win_actual_distances), np.nanmean(player_1_win_direct_distances/player_1_win_actual_distances)

(np.float64(0.9196845327488561), np.float64(0.9753591794777994))

In [571]:
np.nanmean(player_1_loss_direct_distances/player_1_loss_actual_distances), np.nanmean(player_0_loss_direct_distances/player_0_loss_actual_distances)

C:\Users\Tom\AppData\Local\Temp\ipykernel_16692\3291096355.py:1: RuntimeWarning: invalid value encountered in divide
  np.nanmean(player_1_loss_direct_distances/player_1_loss_actual_distances), np.nanmean(player_0_loss_direct_distances/player_0_loss_actual_distances)


(np.float64(0.9292355321249816), np.float64(0.7639445690966866))

In [572]:
def ratio_direct_to_absolute_distances_session(trial_list):
    ''' Return 4 values, the ratio of direct distances to absolute distances for 
        winner and then loser for first player 0 and then player 1 '''
    
    

In [573]:
player_0_loss_actual_distances

array([ 4.28021165,         nan, 20.67565703,         nan,         nan,
               nan, 14.56262786,         nan,         nan, 11.81873951,
       18.81694117,         nan, 15.90104251, 16.91283744, 20.11940893,
       17.2323116 , 28.9369385 , 14.71371709,         nan, 18.54751341,
       16.38583798,         nan, 24.40519673, 15.72767363,         nan,
               nan, 18.80061008, 17.88969583, 17.79182371, 17.58255119,
       19.31937395, 24.09865682,         nan,         nan, 16.84412562,
       20.68561902,         nan, 18.14526538,         nan,         nan,
       17.88344734,  4.63191192, 22.48297362, 15.23346828,         nan,
               nan,         nan,         nan,         nan, 17.17867976,
       18.33212597,  9.48644216, 17.16822551,         nan, 19.44517436,
               nan, 26.58286675,         nan, 10.72368652, 25.61150888,
       22.83831301, 19.46566728,         nan,         nan, 17.04338638,
       25.03698294, 15.71464681, 16.88784556,         nan, 18.38

In [574]:
get_indices.get_trigger_activators(trial_list)

array([1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1.])

In [575]:
a = player_actual_distance_session(trial_list, 1)
b = player_direct_distance_session(trial_list, 1)

In [576]:
a

array([2.52305450e+01, 3.16390224e+01, 1.98478816e+01, 2.41265566e+01,
       1.13575656e+01, 1.93331217e+01, 2.18866825e+01, 1.94101196e+01,
       1.94708992e+01, 1.96996371e+01, 1.76295867e+01, 3.69084240e+00,
       1.97388398e+01, 1.75607818e+01, 1.74153651e+01, 1.72987599e+01,
       3.42576149e+01, 1.92115005e+01, 1.68618933e+01, 1.62915807e+01,
       1.74928967e+01, 1.46644295e+01, 1.88723422e+01, 1.85705133e+01,
       2.01810424e+01, 1.79554402e+01, 1.93790806e+01, 1.63122159e+01,
       1.74285263e+01, 1.58826633e+01, 1.73459946e+01, 1.68992438e+01,
       8.33544534e+00, 1.71380538e+01, 1.85821888e+01, 1.86433885e+01,
       9.43097352e+00, 1.68317588e+01, 1.29956546e+01, 1.77288677e+01,
       1.57139315e+01, 1.88428620e+01, 1.86418370e+01, 1.90495324e+01,
       8.62903881e+00, 1.68587394e-07, 1.50548841e+01, 1.38228209e+01,
       1.54696057e+01, 1.66384707e+01, 1.76936575e+01, 1.61997005e+01,
       1.74264977e+01, 1.50679193e+01, 1.90154105e+01, 1.72695766e+01,
      